In [1]:
%pip install openpyxl
%pip install shap
%pip install xgboost
%pip install torch
%pip install matplotlib
%pip install seaborn
%pip install scipy
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import shap
import joblib
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from scipy import stats
import json
from datetime import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

In [2]:
from nam_analysis import NAMAnalyzer

In [4]:
FEATURE_DICT = {
    'lw': 'Length',
    'hw': 'Height',
    'tw': 'Thickness',
    'f′c': 'Concrete compressive strength',
    'fyt': 'Transverse web reinforcement yield strength',
    'fysh': 'Transverse boundary reinforcement yield strength',
    'fyl': 'Vertical web reinforcement yield strength',
    'fybl': 'Vertical boundary reinforcement yield strength',
    'ρt': 'Transverse web reinforcement ratio',
    'ρsh': 'Transverse boundary reinforcement ratio',
    'ρl': 'Vertical web reinforcement ratio',
    'ρbl': 'Vertical boundary reinforcement ratio',
    'P/(Agf′c)': 'Axial Load Ratio',
    'b0': 'Boundary element depth',
    'db': 'Boundary element length',
    's/db': 'Hoop spacing / Boundary element length',
    'AR': 'Aspect ratio',
    'M/Vlw': 'Shear span ratio'
}

In [6]:
def sanitize_filename(filename):
    """Convert a string into a valid filename by removing invalid characters."""
    # Replace invalid characters with underscores
    filename = re.sub(r'[/\\?%*:|"<>()\'{}]', '_', filename)
    # Replace Greek letters and special characters
    filename = filename.replace('ρ', 'rho')
    filename = filename.replace('′', '_prime_')
    # Remove multiple underscores
    filename = re.sub(r'_+', '_', filename)
    # Remove leading/trailing underscores
    filename = filename.strip('_')
    return filename

def analyze_and_transform_skewed_features(data, features=['fysh', 'ρsh'], threshold=1.0, high_skew_threshold=3.0):
    """
    Analyze and transform skewed features using various transformations.
    """
    transformed_data = data.copy()
    transformations = {}

    for feature in features:
        print(f"\n{feature}:")
        # Calculate skewness
        original_skew = data[feature].skew()
        print(f"  Original skewness: {original_skew}")

        # Apply transformations if skewed
        if abs(original_skew) > threshold:
            # Prepare data
            min_value = data[feature].min()
            max_value = data[feature].max()
            range_value = max_value - min_value
            offset = -min_value + range_value * 0.01

            print("  Transformation results:")
            
            # Try different transformations
            transformations_to_try = []

            # 1. Log transformation
            log_data = np.log1p(data[feature] + offset)
            log_skew = log_data.skew()
            print(f"    log1p: {log_skew:.2f}")
            transformations_to_try.append(('log1p', log_data, log_skew))

            # 2. Power transformation (x^0.2)
            power_data = np.power(data[feature] + offset, 0.2)
            power_skew = power_data.skew()
            print(f"    power: {power_skew:.2f}")
            transformations_to_try.append(('power', power_data, power_skew))

            # 3. Rank (percentile) transformation
            rank_data = data[feature].rank(pct=True)
            rank_skew = rank_data.skew()
            print(f"    rank: {rank_skew:.2f}")
            transformations_to_try.append(('rank', rank_data, rank_skew))

            # Choose the best transformation (lowest absolute skewness)
            best_transform = min(transformations_to_try, key=lambda x: abs(x[2]))
            transform_type, transformed_values, new_skew = best_transform
            print(f"  Selected: {transform_type} (skewness: {new_skew:.2f})")

            # Apply the best transformation
            transformed_data[feature] = transformed_values

            transformations[feature] = {
                'type': transform_type,
                'original_skew': original_skew,
                'transformed_skew': new_skew
            }

    return transformed_data, transformations


def load_and_process_data(excel_path="Database_ShearWall.xlsx", verbose=True, test_size=0.2, random_state=42):
    """
    Load and process the shear wall database.

    Parameters:
    -----------
    excel_path : str, default="Database_ShearWall.xlsx"
        Path to the Excel file containing the database
    verbose : bool, default=True
        Whether to print information about the data
    test_size : float, default=0.2
        Proportion of the dataset to include in the test split
    random_state : int, default=42
        Random state for reproducibility

    Returns:
    --------
    dict
        Dictionary containing:
        - X_train, X_test: Training/testing features (normalized to [-1, 1])
        - y_train, y_test: Training/testing targets (normalized to [-1, 1])
        - feature_names: List of feature names
        - x_scaler: Fitted MinMaxScaler for features
        - y_scaler: Fitted MinMaxScaler for target
        - transformations: Dictionary containing transformation info
    """
    # Input validation
    if not os.path.exists(excel_path):
        raise FileNotFoundError(f"Excel file not found: {excel_path}")

    # List of features in order
    feature_names = list(FEATURE_DICT.keys())
    output_name = 'NCDE'  # The target variable

    try:
        # Read the database, skipping the first 3 rows which are headers
        df = pd.read_excel(excel_path, skiprows=3)

        if verbose:
            print("Original dataframe shape:", df.shape)

        # Remove the metadata columns (first 3 columns) and the last 3 columns
        df_processed = df.iloc[:, 3:-3]

        # Validate number of columns
        expected_cols = len(feature_names) + 1  # features + NCDE
        if df_processed.shape[1] != expected_cols:
            raise ValueError(f"Expected {expected_cols} columns but got {df_processed.shape[1]}")

        # Extract features and target
        X = df_processed.iloc[:, :18]
        y = df_processed.iloc[:, 18:19]  # NCDE column

        # Rename columns for clarity
        X.columns = feature_names
        y.columns = [output_name]

        # Convert data to numeric, replacing '-' with NaN
        X = X.replace({'-': None}).astype(float)
        y = y.astype(float)

        # Print information about missing values if verbose
        if verbose:
            print("\nMissing values in features:")
            missing_values = X.isnull().sum()
            for feature, count in missing_values.items():
                if count > 0:
                    print(f"{feature:10} : {count} ({count/len(X)*100:.1f}%)")

        # Validate target variable
        if y.isnull().any().any():
            raise ValueError("Target variable (NCDE) contains missing values")

    except Exception as e:
        raise Exception(f"Error loading data: {str(e)}")

    # Analyze and transform skewed features
    skewed_features = ['fysh', 'ρsh']
    X_transformed, transformations = analyze_and_transform_skewed_features(X, features=skewed_features)

    if verbose:
        print("\nFeature Transformations:")
        for feature, info in transformations.items():
            if info['type'] == 'log1p':
                print(f"{feature}:")
                print(f"  Original skewness: {info['original_skew']:.2f}")
                print(f"  After log transform: {info['transformed_skew']:.2f}")

    # Analyze ρsh distribution before any transformations
    print("\nAnalyzing ρsh distribution before transformation:")
    rho_sh_stats = analyze_feature_distribution(X, 'ρsh')

    # Split the transformed data
    X_train, X_test, y_train, y_test = train_test_split(
        X_transformed, y, test_size=test_size, random_state=random_state
    )

    # Compute medians from training data only
    train_medians = X_train.median()

    # Fill missing values in both sets using training medians
    X_train_filled = X_train.fillna(train_medians)
    X_test_filled = X_test.fillna(train_medians)

    # Initialize and fit scalers on training data only
    x_scaler = MinMaxScaler(feature_range=(-1, 1))
    y_scaler = MinMaxScaler(feature_range=(-1, 1))

    # Fit scalers on training data and transform both sets
    X_train_normalized = pd.DataFrame(
        x_scaler.fit_transform(X_train_filled),
        columns=X_train_filled.columns,
        index=X_train_filled.index
    )

    X_test_normalized = pd.DataFrame(
        x_scaler.transform(X_test_filled),
        columns=X_test_filled.columns,
        index=X_test_filled.index
    )

    y_train_normalized = pd.DataFrame(
        y_scaler.fit_transform(y_train),
        columns=y_train.columns,
        index=y_train.index
    )

    y_test_normalized = pd.DataFrame(
        y_scaler.transform(y_test),
        columns=y_test.columns,
        index=y_test.index
    )

    if verbose:
        print("\nFeature Dictionary (Abbreviation -> Full Name):")
        for abbr, full_name in FEATURE_DICT.items():
            print(f"{abbr:10} : {full_name}")

        print("\nData shapes:")
        print("Training set (X_train):", X_train_normalized.shape)
        print("Testing set (X_test):", X_test_normalized.shape)
        print("Training labels (y_train):", y_train_normalized.shape)
        print("Testing labels (y_test):", y_test_normalized.shape)

        print("\nFeature ranges after normalization (training set):")
        for col in X_train_normalized.columns:
            print(f"{col:10} : [{X_train_normalized[col].min():.2f}, {X_train_normalized[col].max():.2f}]")

        print("\nTarget range after normalization (training set):")
        print(f"NCDE      : [{y_train_normalized[output_name].min():.2f}, {y_train_normalized[output_name].max():.2f}]")

        print("\nFirst few rows of normalized training data:")
        print(X_train_normalized.head())
        print("\nFirst few rows of normalized training labels (NCDE):")
        print(y_train_normalized.head())

    return {
        'X_train': X_train_normalized,
        'X_test': X_test_normalized,
        'y_train': y_train_normalized,
        'y_test': y_test_normalized,
        'feature_names': feature_names,
        'x_scaler': x_scaler,
        'y_scaler': y_scaler,
        'transformations': transformations
    }

def analyze_features(data_dict, save_plots=True, output_dir="plots"):
    """
    Perform comprehensive analysis of features including distributions,
    correlations, and basic statistics.

    Parameters:
    -----------
    data_dict : dict
        Dictionary containing X, y, and metadata from load_and_process_data()
    save_plots : bool, default=True
        Whether to save plots to files
    output_dir : str, default="plots"
        Directory to save plots if save_plots is True
    """
    # Extract X and y from the data dictionary
    X = pd.concat([data_dict['X_train'], data_dict['X_test']])
    y = pd.concat([data_dict['y_train'], data_dict['y_test']])
    feature_names = data_dict['feature_names']

    # Create output directory if it doesn't exist
    if save_plots and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Convert any string values to numeric, replacing non-numeric with NaN
    X = X.apply(pd.to_numeric, errors='coerce')

    # Basic statistics
    stats_df = pd.DataFrame({
        'mean': X.mean(),
        'std': X.std(),
        'min': X.min(),
        'max': X.max(),
        'skew': X.skew(),
        'kurtosis': X.kurtosis()
    })
    print("\nFeature Statistics:")
    print(stats_df)

    # Create box plots for all features
    plt.figure(figsize=(15, 8))
    sns.boxplot(data=X)
    plt.xticks(rotation=45, ha='right')
    plt.title('Box Plots of All Features')
    plt.tight_layout()
    if save_plots:
        plt.savefig(f"{output_dir}/feature_boxplots.png", bbox_inches='tight', dpi=300)
    plt.close()

    # Create statistical summary plots
    for feature in feature_names:
        plt.figure(figsize=(10, 6))

        # Create bar plot for statistics
        stats_values = [
            stats_df.loc[feature, 'mean'],
            stats_df.loc[feature, 'std'],
            stats_df.loc[feature, 'min'],
            stats_df.loc[feature, 'max']
        ]
        stats_labels = ['Mean', 'Std Dev', 'Min', 'Max']

        # Create bar plot
        bars = plt.bar(stats_labels, stats_values)

        # Add value labels on top of bars
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.2f}',
                    ha='center', va='bottom')

        plt.title(f'Statistical Summary of {feature}\n({FEATURE_DICT[feature]})')
        plt.ylabel('Value')

        if save_plots:
            safe_filename = sanitize_filename(feature)
            plt.savefig(f"{output_dir}/stats_summary_{safe_filename}.png", bbox_inches='tight', dpi=300)
        plt.close()

    # Create a normalized parallel coordinates plot
    plt.figure(figsize=(15, 8))
    # Normalize the data for better visualization
    X_norm = (X - X.min()) / (X.max() - X.min())
    pd.plotting.parallel_coordinates(
        pd.concat([X_norm, pd.Series(np.zeros(len(X)), name='group')], axis=1),
        'group',
        alpha=0.1
    )
    plt.xticks(rotation=45, ha='right')
    plt.title('Normalized Parallel Coordinates Plot of All Features')
    plt.tight_layout()
    if save_plots:
        plt.savefig(f"{output_dir}/parallel_coordinates.png", bbox_inches='tight', dpi=300)
    plt.close()

    # Create statistical overview heatmap
    # Normalize statistics for better visualization
    stats_for_heatmap = stats_df.copy()
    for col in stats_for_heatmap.columns:
        stats_for_heatmap[col] = (stats_for_heatmap[col] - stats_for_heatmap[col].min()) / \
                                (stats_for_heatmap[col].max() - stats_for_heatmap[col].min())

    plt.figure(figsize=(12, 10))
    sns.heatmap(stats_for_heatmap.T, annot=True, fmt='.2f', cmap='YlOrRd')
    plt.title('Statistical Measures Heatmap (Normalized)')
    plt.tight_layout()
    if save_plots:
        plt.savefig(f"{output_dir}/statistics_heatmap.png", bbox_inches='tight', dpi=300)
    plt.close()

    # Individual Feature Distributions
    for feature in feature_names:
        plt.figure(figsize=(10, 6))
        sns.histplot(X[feature].dropna(), kde=True)
        plt.title(f'Distribution of {feature}')
        plt.xlabel(FEATURE_DICT[feature])
        if save_plots:
            safe_filename = sanitize_filename(feature)
            plt.savefig(f"{output_dir}/distribution_{safe_filename}.png", bbox_inches='tight', dpi=300)
        plt.close()

    # NCDE Distribution
    plt.figure(figsize=(10, 6))
    sns.histplot(y['NCDE'], kde=True)
    plt.title('Distribution of NCDE (Output)')
    plt.xlabel('NCDE')
    if save_plots:
        plt.savefig(f"{output_dir}/distribution_NCDE_output.png", bbox_inches='tight', dpi=300)
    plt.close()

    # Correlations
    # Features vs NCDE
    feature_ncde_corr = pd.DataFrame(index=feature_names, columns=['correlation'])
    for feature in feature_names:
        # Drop any NaN values for correlation calculation
        valid_data = pd.concat([X[feature], y['NCDE']], axis=1).dropna()
        if not valid_data.empty:
            feature_ncde_corr.loc[feature, 'correlation'] = stats.pearsonr(
                valid_data[feature],
                valid_data['NCDE']
            )[0]

    # Plot feature-NCDE correlations
    plt.figure(figsize=(12, 6))
    sns.barplot(x=feature_ncde_corr.index, y='correlation', data=feature_ncde_corr)
    plt.xticks(rotation=45, ha='right')
    plt.title('Feature Correlations with NCDE')
    plt.tight_layout()
    if save_plots:
        plt.savefig(f"{output_dir}/correlations_features_vs_NCDE.png", bbox_inches='tight', dpi=300)
    plt.close()

    # Inter-feature correlations heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(X.corr(), annot=True, cmap='coolwarm', center=0, fmt='.2f')
    plt.title('Inter-feature Correlation Matrix')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    if save_plots:
        plt.savefig(f"{output_dir}/correlations_heatmap.png", bbox_inches='tight', dpi=300)
    plt.close()

    # Save statistics to CSV
    if save_plots:
        stats_df.to_csv(f"{output_dir}/feature_statistics.csv")
        feature_ncde_corr.to_csv(f"{output_dir}/feature_ncde_correlations.csv")

    # Identify highly correlated features
    corr_matrix = X.corr()
    high_corr_threshold = 0.7
    high_corr_pairs = []
    for i in range(len(feature_names)):
        for j in range(i+1, len(feature_names)):
            corr = abs(corr_matrix.iloc[i, j])
            if corr > high_corr_threshold and not np.isnan(corr):
                high_corr_pairs.append((
                    feature_names[i],
                    feature_names[j],
                    corr
                ))

    if high_corr_pairs:
        print("\nHighly correlated feature pairs (|r| > 0.7):")
        for f1, f2, corr in high_corr_pairs:
            print(f"{f1} - {f2}: {corr:.3f}")

    return {
        'statistics': stats_df,
        'feature_ncde_correlations': feature_ncde_corr,
        'correlation_matrix': corr_matrix,
        'high_correlation_pairs': high_corr_pairs
    }

def analyze_feature_distribution(data, feature_name):
    """Analyze the distribution of a specific feature in detail."""
    # Get values and remove NaN values
    values = data[feature_name].dropna()

    # Basic statistics
    stats_dict = {
        'count': len(values),
        'mean': values.mean(),
        'median': values.median(),
        'std': values.std(),
        'min': values.min(),
        'max': values.max(),
        'skew': values.skew(),
        'kurtosis': values.kurtosis(),
        'zeros': (values == 0).sum(),
        'unique_values': len(values.unique()),
        'missing_values': data[feature_name].isna().sum()
    }

    # Calculate percentiles
    percentiles = [0, 1, 5, 10, 25, 50, 75, 90, 95, 99, 100]
    for p in percentiles:
        stats_dict[f'p{p}'] = np.nanpercentile(values, p)

    # Print results
    print(f"\nDetailed Analysis of {feature_name}:")
    print(f"Count: {stats_dict['count']}")
    print(f"Missing values: {stats_dict['missing_values']}")
    print(f"Unique values: {stats_dict['unique_values']}")
    print(f"Zeros: {stats_dict['zeros']} ({stats_dict['zeros']/stats_dict['count']*100:.1f}%)")
    print(f"\nCentral Tendency:")
    print(f"Mean: {stats_dict['mean']:.6f}")
    print(f"Median: {stats_dict['median']:.6f}")
    print(f"Std Dev: {stats_dict['std']:.6f}")
    print(f"\nShape:")
    print(f"Skewness: {stats_dict['skew']:.2f}")
    print(f"Kurtosis: {stats_dict['kurtosis']:.2f}")
    print(f"\nRange:")
    print(f"Min: {stats_dict['min']:.6f}")
    print(f"Max: {stats_dict['max']:.6f}")

    print("\nPercentiles:")
    for p in percentiles:
        print(f"p{p}: {stats_dict[f'p{p}']:.6f}")

    # Create visualization
    plt.figure(figsize=(15, 5))

    # Histogram with density
    plt.subplot(131)
    sns.histplot(data=values, kde=True)
    plt.title(f'Distribution of {feature_name}')
    plt.xlabel(feature_name)

    # Box plot
    plt.subplot(132)
    sns.boxplot(y=values)
    plt.title(f'Box Plot of {feature_name}')

    # Q-Q plot
    plt.subplot(133)
    from scipy import stats as scipy_stats
    scipy_stats.probplot(values, dist="norm", plot=plt)
    plt.title(f'Q-Q Plot of {feature_name}')

    plt.tight_layout()
    plt.savefig(f'feature_analysis/{feature_name}_detailed_analysis.png')
    plt.close()

    return stats_dict

# If the script is run directly, show the example usage
if __name__ == "__main__":
    # Get the data in the new format
    data_dict = load_and_process_data(verbose=True)

    # Run the feature analysis
    analysis = analyze_features(data_dict)

Original dataframe shape: (312, 25)

Missing values in features:
fysh       : 100 (32.1%)
fybl       : 29 (9.3%)
ρsh        : 96 (30.8%)
ρbl        : 32 (10.3%)

fysh:
  Original skewness: 2.1691698429431523
  Transformation results:
    log1p: -1.37
    power: -0.41
    rank: 0.00
  Selected: rank (skewness: 0.00)

ρsh:
  Original skewness: 5.072272455267313
  Transformation results:
    log1p: 4.87
    power: 0.43
    rank: 0.00
  Selected: rank (skewness: 0.00)

Feature Transformations:

Analyzing ρsh distribution before transformation:

Detailed Analysis of ρsh:
Count: 216
Missing values: 96
Unique values: 113
Zeros: 0 (0.0%)

Central Tendency:
Mean: 0.009859
Median: 0.008177
Std Dev: 0.009038

Shape:
Skewness: 5.07
Kurtosis: 40.30

Range:
Min: 0.000950
Max: 0.095700

Percentiles:
p0: 0.000950
p1: 0.001340
p5: 0.002248
p10: 0.002513
p25: 0.005309
p50: 0.008177
p75: 0.011166
p90: 0.016956
p95: 0.023020
p99: 0.042834
p100: 0.095700

Feature Dictionary (Abbreviation -> Full Name):
lw 

Training NAMs and picking the best one

In [7]:
def optimize_nam():
    # Set random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)

    print("1. Loading data...")
    data_dict = load_and_process_data(verbose=False)

    print("\n2. Initializing NAM analyzer...")
    analyzer = NAMAnalyzer(data_dict)

    print("\n3. Running grid search...")
    # Define parameter grid for the new NAM architecture
    param_grid = {
        'hidden_layers': [
            [64, 128],      # Two layers: medium -> large
            [128, 64],      # Two layers: large -> medium (bottleneck)
            [64],           # Single medium layer
            [128]           # Single large layer
        ],
        'learning_rate': [0.001, 0.0005],
        'batch_size': [32, 64]
    }

    results = analyzer.grid_search(param_grid)

    print("\n4. Analyzing best model...")
    analyzer.analyze_best_model()

    print("\nOptimization completed successfully!")

optimize_nam()

1. Loading data...

fysh:
  Original skewness: 2.1691698429431523
  Transformation results:
    log1p: -1.37
    power: -0.41
    rank: 0.00
  Selected: rank (skewness: 0.00)

ρsh:
  Original skewness: 5.072272455267313
  Transformation results:
    log1p: 4.87
    power: 0.43
    rank: 0.00
  Selected: rank (skewness: 0.00)

Analyzing ρsh distribution before transformation:

Detailed Analysis of ρsh:
Count: 216
Missing values: 96
Unique values: 113
Zeros: 0 (0.0%)

Central Tendency:
Mean: 0.009859
Median: 0.008177
Std Dev: 0.009038

Shape:
Skewness: 5.07
Kurtosis: 40.30

Range:
Min: 0.000950
Max: 0.095700

Percentiles:
p0: 0.000950
p1: 0.001340
p5: 0.002248
p10: 0.002513
p25: 0.005309
p50: 0.008177
p75: 0.011166
p90: 0.016956
p95: 0.023020
p99: 0.042834
p100: 0.095700

2. Initializing NAM analyzer...
Using device: cpu

3. Running grid search...

Testing configuration 1/16:
{
  "hidden_layers": [
    64,
    128
  ],
  "learning_rate": 0.001,
  "batch_size": 32
}


: 

Training Baseline

In [ ]:
def train_and_evaluate_model(model, X_train, X_test, y_train, y_test, y_scaler):
    """Train a model and evaluate its performance."""
    # Train the model
    model.fit(X_train, y_train.values.ravel())

    # Make predictions
    y_pred = model.predict(X_test)

    # Convert predictions back to original scale
    y_pred_orig = y_scaler.inverse_transform(y_pred.reshape(-1, 1)).reshape(-1)
    y_true_orig = y_scaler.inverse_transform(y_test.values).reshape(-1)

    # Calculate metrics
    metrics = {
        'rmse': float(np.sqrt(mean_squared_error(y_true_orig, y_pred_orig))),
        'mae': float(mean_absolute_error(y_true_orig, y_pred_orig)),
        'r2': float(r2_score(y_true_orig, y_pred_orig)),
        'explained_variance': float(explained_variance_score(y_true_orig, y_pred_orig))
    }

    return metrics

def optimize_random_forest():
    """Optimize Random Forest model."""
    param_grid = [
        {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 2},
        {'n_estimators': 200, 'max_depth': 15, 'min_samples_split': 5},
        {'n_estimators': 300, 'max_depth': None, 'min_samples_split': 10}
    ]

    results = []
    best_metrics = {'r2': float('-inf')}
    best_model = None

    for params in param_grid:
        model = RandomForestRegressor(
            random_state=42,
            n_jobs=-1,
            **params
        )
        metrics = train_and_evaluate_model(model, X_train, X_test, y_train, y_test, y_scaler)

        result = {
            'model_type': 'RandomForest',
            'params': params,
            'metrics': metrics
        }
        results.append(result)

        if metrics['r2'] > best_metrics['r2']:
            best_metrics = metrics
            best_model = model

    return results, best_model

def optimize_xgboost():
    """Optimize XGBoost model."""
    param_grid = [
        {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1},
        {'n_estimators': 200, 'max_depth': 8, 'learning_rate': 0.05},
        {'n_estimators': 500, 'max_depth': 30, 'learning_rate': 0.01}
    ]

    results = []
    best_metrics = {'r2': float('-inf')}
    best_model = None

    for params in param_grid:
        model = XGBRegressor(
            random_state=42,
            n_jobs=-1,
            **params
        )
        metrics = train_and_evaluate_model(model, X_train, X_test, y_train, y_test, y_scaler)

        result = {
            'model_type': 'XGBoost',
            'params': params,
            'metrics': metrics
        }
        results.append(result)

        if metrics['r2'] > best_metrics['r2']:
            best_metrics = metrics
            best_model = model

    return results, best_model

if __name__ == "__main__":
    # Load data
    print("Loading data...")
    data_dict = load_and_process_data(verbose=False)

    # Extract data
    X_train = data_dict['X_train']
    X_test = data_dict['X_test']
    y_train = data_dict['y_train']
    y_test = data_dict['y_test']
    y_scaler = data_dict['y_scaler']

    # Create results directory
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    results_dir = f'baseline_results_{timestamp}'
    os.makedirs(results_dir, exist_ok=True)

    # Optimize Random Forest
    print("\nOptimizing Random Forest...")
    rf_results, best_rf = optimize_random_forest()

    # Optimize XGBoost
    print("\nOptimizing XGBoost...")
    xgb_results, best_xgb = optimize_xgboost()

    # Combine results
    all_results = rf_results + xgb_results

    # Save results
    print("\nSaving results...")
    results_file = f'{results_dir}/baseline_results.json'
    with open(results_file, 'w') as f:
        json.dump(all_results, f, indent=2)

    # Save best models
    joblib.dump(best_rf, f'{results_dir}/best_random_forest.joblib')
    joblib.dump(best_xgb, f'{results_dir}/best_xgboost.joblib')

    # Print best results
    print("\nRandom Forest Results:")
    for result in rf_results:
        print(f"\nParameters: {result['params']}")
        print(f"Metrics: {result['metrics']}")

    print("\nXGBoost Results:")
    for result in xgb_results:
        print(f"\nParameters: {result['params']}")
        print(f"Metrics: {result['metrics']}")

    print(f"\nResults and models saved in: {results_dir}")

In [ ]:
import json
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from analyze_best_models import find_latest_results_dir as find_nam_dir

def find_latest_baseline_dir():
    """Find the most recent baseline results directory."""
    dirs = [d for d in os.listdir('.') if d.startswith('baseline_results_')]
    if not dirs:
        raise ValueError("No baseline results directory found!")
    return sorted(dirs)[-1]

def load_best_models():
    """Load the best models and their results."""
    # Load NAM results
    nam_dir = find_nam_dir()
    with open(f'{nam_dir}/all_results.json', 'r') as f:
        nam_results = json.load(f)

    # Handle different metrics structure in NAM results
    def get_metrics(result):
        if 'test_metrics' in result['metrics']:  # Old NAM format
            return result['metrics']['test_metrics']
        return result['metrics']  # New format (baselines)

    best_nam = sorted(nam_results, key=lambda x: get_metrics(x)['r2'], reverse=True)[0]

    # Load baseline results
    baseline_dir = find_latest_baseline_dir()
    with open(f'{baseline_dir}/baseline_results.json', 'r') as f:
        baseline_results = json.load(f)

    # Load best baseline models
    best_rf = joblib.load(f'{baseline_dir}/best_random_forest.joblib')
    best_xgb = joblib.load(f'{baseline_dir}/best_xgboost.joblib')

    # Get best results for each model type
    rf_results = [r for r in baseline_results if r['model_type'] == 'RandomForest']
    xgb_results = [r for r in baseline_results if r['model_type'] == 'XGBoost']
    best_rf_result = sorted(rf_results, key=lambda x: x['metrics']['r2'], reverse=True)[0]
    best_xgb_result = sorted(xgb_results, key=lambda x: x['metrics']['r2'], reverse=True)[0]

    return {
        'NAM': {'result': {'params': best_nam['params'], 'metrics': get_metrics(best_nam)}},
        'RandomForest': {'model': best_rf, 'result': best_rf_result},
        'XGBoost': {'model': best_xgb, 'result': best_xgb_result}
    }

def plot_model_comparison(models_dict):
    """Create comparison plots for all models."""
    # Set up the plot style
    plt.rcParams['figure.figsize'] = (15, 10)
    plt.rcParams['font.size'] = 10

    # Create figure
    fig = plt.figure()

    # Extract metrics
    model_names = list(models_dict.keys())
    metrics = {
        'R²': [models_dict[m]['result']['metrics']['r2'] for m in model_names],
        'RMSE': [models_dict[m]['result']['metrics']['rmse'] for m in model_names],
        'MAE': [models_dict[m]['result']['metrics']['mae'] for m in model_names],
        'Explained Variance': [models_dict[m]['result']['metrics']['explained_variance'] for m in model_names]
    }

    # Color scheme
    colors = ['#2ecc71', '#3498db', '#e74c3c']

    # Create subplots for each metric
    for i, (metric_name, values) in enumerate(metrics.items(), 1):
        plt.subplot(2, 2, i)
        bars = plt.bar(model_names, values, color=colors)
        plt.title(f'{metric_name} Comparison', pad=20)
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)

        # Add value labels on top of bars
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.4f}',
                    ha='center', va='bottom')

    plt.tight_layout()
    plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

def print_detailed_comparison(models_dict):
    """Print detailed comparison of all models."""
    print("\n=== Model Comparison ===")
    print("\nBest Model Configurations:")

    for model_name, model_info in models_dict.items():
        print(f"\n{model_name}:")
        print("Parameters:")
        print(json.dumps(model_info['result']['params'], indent=2))
        print("\nMetrics:")
        metrics = model_info['result']['metrics']
        print(f"R²: {metrics['r2']:.4f}")
        print(f"RMSE: {metrics['rmse']:.4f}")
        print(f"MAE: {metrics['mae']:.4f}")
        print(f"Explained Variance: {metrics['explained_variance']:.4f}")

    # Find the best model
    best_model = max(models_dict.items(), key=lambda x: x[1]['result']['metrics']['r2'])
    print(f"\n🏆 Best Overall Model: {best_model[0]}")
    print(f"R² Score: {best_model[1]['result']['metrics']['r2']:.4f}")

def main():
    print("Loading best models and results...")
    models_dict = load_best_models()

    print("\nCreating comparison plots...")
    plot_model_comparison(models_dict)

    print("\nGenerating detailed comparison...")
    print_detailed_comparison(models_dict)

    print("\nComparison plot saved as 'model_comparison.png'")

if __name__ == "__main__":
    main()

In [ ]:
import json
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from analyze_database import load_and_process_data
from analyze_best_models import find_latest_results_dir as find_nam_dir
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import torch
from nam_analysis import NAM
from analyze_database import FEATURE_DICT

def find_latest_baseline_dir():
    """Find the most recent baseline results directory."""
    dirs = [d for d in os.listdir('.') if d.startswith('baseline_results_')]
    if not dirs:
        raise ValueError("No baseline results directory found!")
    return sorted(dirs)[-1]

def load_best_models():
    """Load the best models and their results."""
    # Load NAM results
    nam_dir = find_nam_dir()
    with open(f'{nam_dir}/all_results.json', 'r') as f:
        nam_results = json.load(f)

    # Handle different metrics structure in NAM results
    def get_metrics(result):
        if 'test_metrics' in result['metrics']:  # Old NAM format
            return result['metrics']['test_metrics']
        return result['metrics']  # New format (baselines)

    best_nam = sorted(nam_results, key=lambda x: get_metrics(x)['r2'], reverse=True)[0]

    # Load baseline results
    baseline_dir = find_latest_baseline_dir()
    with open(f'{baseline_dir}/baseline_results.json', 'r') as f:
        baseline_results = json.load(f)

    # Load best baseline models
    best_rf = joblib.load(f'{baseline_dir}/best_random_forest.joblib')
    best_xgb = joblib.load(f'{baseline_dir}/best_xgboost.joblib')

    # Get best results for each model type
    rf_results = [r for r in baseline_results if r['model_type'] == 'RandomForest']
    xgb_results = [r for r in baseline_results if r['model_type'] == 'XGBoost']
    best_rf_result = sorted(rf_results, key=lambda x: x['metrics']['r2'], reverse=True)[0]
    best_xgb_result = sorted(xgb_results, key=lambda x: x['metrics']['r2'], reverse=True)[0]

    return {
        'NAM': {'result': {'params': best_nam['params'], 'metrics': get_metrics(best_nam)}},
        'RandomForest': {'model': best_rf, 'result': best_rf_result},
        'XGBoost': {'model': best_xgb, 'result': best_xgb_result}
    }

def plot_model_comparison(models_dict):
    """Create comparison plots for all models."""
    # Set up the plot style
    plt.rcParams['figure.figsize'] = (15, 10)
    plt.rcParams['font.size'] = 10

    # Create figure
    fig = plt.figure()

    # Extract metrics
    model_names = list(models_dict.keys())
    metrics = {
        'R²': [models_dict[m]['result']['metrics']['r2'] for m in model_names],
        'RMSE': [models_dict[m]['result']['metrics']['rmse'] for m in model_names],
        'MAE': [models_dict[m]['result']['metrics']['mae'] for m in model_names],
        'Explained Variance': [models_dict[m]['result']['metrics']['explained_variance'] for m in model_names]
    }

    # Color scheme
    colors = ['#2ecc71', '#3498db', '#e74c3c']

    # Create subplots for each metric
    for i, (metric_name, values) in enumerate(metrics.items(), 1):
        plt.subplot(2, 2, i)
        bars = plt.bar(model_names, values, color=colors)
        plt.title(f'{metric_name} Comparison', pad=20)
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)

        # Add value labels on top of bars
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.4f}',
                    ha='center', va='bottom')

    plt.tight_layout()
    plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

def print_detailed_comparison(models_dict):
    """Print detailed comparison of all models."""
    print("\n=== Model Comparison ===")
    print("\nBest Model Configurations:")

    for model_name, model_info in models_dict.items():
        print(f"\n{model_name}:")
        print("Parameters:")
        print(json.dumps(model_info['result']['params'], indent=2))
        print("\nMetrics:")
        metrics = model_info['result']['metrics']
        print(f"R²: {metrics['r2']:.4f}")
        print(f"RMSE: {metrics['rmse']:.4f}")
        print(f"MAE: {metrics['mae']:.4f}")
        print(f"Explained Variance: {metrics['explained_variance']:.4f}")

    # Find the best model
    best_model = max(models_dict.items(), key=lambda x: x[1]['result']['metrics']['r2'])
    print(f"\n🏆 Best Overall Model: {best_model[0]}")
    print(f"R² Score: {best_model[1]['result']['metrics']['r2']:.4f}")

def load_model_predictions(data_dict):
    """Load predictions from all three models."""
    # Apply transformations to test data if needed
    X_test = data_dict['X_test'].copy()
    if 'transformations' in data_dict:
        for feature, info in data_dict['transformations'].items():
            if info['type'] == 'log1p':
                X_test[feature] = np.log1p(X_test[feature] + info['params']['offset'])
            elif info['type'] == 'power':
                X_test[feature] = np.power(X_test[feature] + info['params']['offset'],
                                         info['params']['power'])
            elif info['type'] == 'rank':
                X_test[feature] = X_test[feature].rank(pct=True)

    # Load NAM predictions
    nam_dirs = [d for d in os.listdir('.') if d.startswith('nam_results_')]
    if not nam_dirs:
        raise FileNotFoundError("No NAM results directory found")
    latest_nam_dir = max(nam_dirs)

    # Load NAM model
    checkpoint = torch.load(os.path.join(latest_nam_dir, 'best_model.pt'), weights_only=False)
    model_state = checkpoint['model_state_dict']
    params = checkpoint['params']

    nam_model = NAM(num_features=len(FEATURE_DICT), hidden_layers=params['hidden_layers'])
    nam_model.load_state_dict(model_state)
    nam_model.eval()

    # Get NAM predictions
    X_test_tensor = torch.FloatTensor(X_test.to_numpy())
    with torch.no_grad():
        nam_preds = nam_model(X_test_tensor)
        if isinstance(nam_preds, tuple):
            nam_preds = nam_preds[0]  # Get the main output if tuple
        nam_preds = nam_preds.cpu().numpy()

    # Find latest baseline directory
    baseline_dirs = [d for d in os.listdir('.') if d.startswith('baseline_results_')]
    if not baseline_dirs:
        raise FileNotFoundError("No baseline results directory found")
    latest_baseline_dir = max(baseline_dirs)

    # Load RF and XGB models from the baseline directory
    rf_model = joblib.load(os.path.join(latest_baseline_dir, 'best_random_forest.joblib'))
    xgb_model = joblib.load(os.path.join(latest_baseline_dir, 'best_xgboost.joblib'))

    # Get RF and XGB predictions
    rf_preds = rf_model.predict(X_test)
    xgb_preds = xgb_model.predict(X_test)

    # Inverse transform predictions to original scale
    y_scaler = data_dict['y_scaler']
    nam_preds = y_scaler.inverse_transform(nam_preds.reshape(-1, 1)).flatten()
    rf_preds = y_scaler.inverse_transform(rf_preds.reshape(-1, 1)).flatten()
    xgb_preds = y_scaler.inverse_transform(xgb_preds.reshape(-1, 1)).flatten()

    # Get true values
    y_true = y_scaler.inverse_transform(data_dict['y_test'].values).flatten()

    return nam_preds, rf_preds, xgb_preds, y_true

def perform_model_anova(nam_preds, rf_preds, xgb_preds, y_true, output_dir="model_analysis"):
    """
    Perform ANOVA and post-hoc tests to compare model predictions.

    Parameters:
    -----------
    nam_preds : array-like
        Predictions from NAM model
    rf_preds : array-like
        Predictions from Random Forest model
    xgb_preds : array-like
        Predictions from XGBoost model
    y_true : array-like
        True target values
    output_dir : str, default="model_analysis"
        Directory to save analysis results
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Calculate residuals
    nam_residuals = y_true - nam_preds
    rf_residuals = y_true - rf_preds
    xgb_residuals = y_true - xgb_preds

    # Calculate performance metrics
    def calc_metrics(y_true, y_pred):
        residuals = y_true - y_pred
        mse = np.mean(residuals ** 2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(residuals))
        r2 = 1 - np.sum(residuals ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)
        return {'RMSE': rmse, 'MAE': mae, 'R²': r2}

    nam_metrics = calc_metrics(y_true, nam_preds)
    rf_metrics = calc_metrics(y_true, rf_preds)
    xgb_metrics = calc_metrics(y_true, xgb_preds)

    # Prepare data for ANOVA
    all_residuals = np.concatenate([nam_residuals, rf_residuals, xgb_residuals])
    model_labels = np.repeat(['NAM', 'RF', 'XGB'], [len(nam_residuals), len(rf_residuals), len(xgb_residuals)])

    # Perform one-way ANOVA
    f_stat, p_value = stats.f_oneway(nam_residuals, rf_residuals, xgb_residuals)

    # Perform Tukey's HSD test
    tukey = pairwise_tukeyhsd(all_residuals, model_labels)

    # Calculate additional statistics
    model_stats = pd.DataFrame({
        'Model': ['NAM', 'RF', 'XGB'],
        'Mean Residual': [np.mean(nam_residuals), np.mean(rf_residuals), np.mean(xgb_residuals)],
        'Std Residual': [np.std(nam_residuals), np.std(rf_residuals), np.std(xgb_residuals)],
        'Mean Abs Residual': [np.mean(np.abs(nam_residuals)),
                            np.mean(np.abs(rf_residuals)),
                            np.mean(np.abs(xgb_residuals))],
        'Median Residual': [np.median(nam_residuals), np.median(rf_residuals), np.median(xgb_residuals)],
        'RMSE': [nam_metrics['RMSE'], rf_metrics['RMSE'], xgb_metrics['RMSE']],
        'MAE': [nam_metrics['MAE'], rf_metrics['MAE'], xgb_metrics['MAE']],
        'R²': [nam_metrics['R²'], rf_metrics['R²'], xgb_metrics['R²']]
    })

    # Create visualization of residuals
    plt.figure(figsize=(12, 6))

    # Box plots
    plt.subplot(1, 2, 1)
    sns.boxplot(x=model_labels, y=all_residuals)
    plt.title('Residual Distribution by Model')
    plt.xlabel('Model')
    plt.ylabel('Residual')

    # Violin plots
    plt.subplot(1, 2, 2)
    sns.violinplot(x=model_labels, y=all_residuals)
    plt.title('Residual Density by Model')
    plt.xlabel('Model')
    plt.ylabel('Residual')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'residual_analysis.png'))
    plt.close()

    # Create Q-Q plots for each model's residuals
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    stats.probplot(nam_residuals, dist="norm", plot=axes[0])
    axes[0].set_title('NAM Q-Q Plot')

    stats.probplot(rf_residuals, dist="norm", plot=axes[1])
    axes[1].set_title('RF Q-Q Plot')

    stats.probplot(xgb_residuals, dist="norm", plot=axes[2])
    axes[2].set_title('XGB Q-Q Plot')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'residual_qq_plots.png'))
    plt.close()

    # Create scatter plots of predicted vs actual values
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    axes[0].scatter(nam_preds, y_true, alpha=0.5)
    axes[0].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--')
    axes[0].set_title(f'NAM (R² = {nam_metrics["R²"]:.3f})')
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('Actual')

    axes[1].scatter(rf_preds, y_true, alpha=0.5)
    axes[1].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--')
    axes[1].set_title(f'Random Forest (R² = {rf_metrics["R²"]:.3f})')
    axes[1].set_xlabel('Predicted')
    axes[1].set_ylabel('Actual')

    axes[2].scatter(xgb_preds, y_true, alpha=0.5)
    axes[2].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--')
    axes[2].set_title(f'XGBoost (R² = {xgb_metrics["R²"]:.3f})')
    axes[2].set_xlabel('Predicted')
    axes[2].set_ylabel('Actual')

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'prediction_scatter_plots.png'))
    plt.close()

    # Generate HTML report
    html_content = f"""
    <html>
    <head>
        <title>Model Comparison Analysis</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 40px; }}
            .section {{ margin: 20px 0; padding: 20px; border: 1px solid #ddd; }}
            .plot {{ text-align: center; margin: 20px 0; }}
            table {{ border-collapse: collapse; width: 100%; margin: 20px 0; }}
            th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
            th {{ background-color: #f5f5f5; }}
            .highlight {{ background-color: #fff3cd; }}
        </style>
    </head>
    <body>
        <h1>Model Comparison Analysis</h1>

        <div class="section">
            <h2>1. Performance Metrics</h2>
            <table>
                <tr>
                    <th>Model</th>
                    <th>RMSE</th>
                    <th>MAE</th>
                    <th>R²</th>
                </tr>
    """

    for _, row in model_stats.iterrows():
        html_content += f"""
                <tr>
                    <td>{row['Model']}</td>
                    <td>{row['RMSE']:.4f}</td>
                    <td>{row['MAE']:.4f}</td>
                    <td>{row['R²']:.4f}</td>
                </tr>
        """

    html_content += """
            </table>
        </div>

        <div class="section">
            <h2>2. ANOVA Results</h2>
            <p>F-statistic: {f_stat:.4f}</p>
            <p>p-value: {p_value:.4f}</p>
            <p>Interpretation: {'There are significant differences between models' if p_value < 0.05
                              else 'No significant differences between models'}</p>
        </div>

        <div class="section">
            <h2>3. Tukey's HSD Test Results</h2>
            <pre>{str(tukey)}</pre>
        </div>

        <div class="section">
            <h2>4. Residual Analysis</h2>
            <table>
                <tr>
                    <th>Model</th>
                    <th>Mean Residual</th>
                    <th>Std Residual</th>
                    <th>Mean Abs Residual</th>
                    <th>Median Residual</th>
                </tr>
    """

    for _, row in model_stats.iterrows():
        html_content += f"""
                <tr>
                    <td>{row['Model']}</td>
                    <td>{row['Mean Residual']:.4f}</td>
                    <td>{row['Std Residual']:.4f}</td>
                    <td>{row['Mean Abs Residual']:.4f}</td>
                    <td>{row['Median Residual']:.4f}</td>
                </tr>
        """

    html_content += """
            </table>
        </div>

        <div class="section">
            <h2>5. Visualization</h2>
            <div class="plot">
                <h3>Residual Analysis</h3>
                <img src="residual_analysis.png" alt="Residual Analysis">
            </div>
            <div class="plot">
                <h3>Q-Q Plots</h3>
                <img src="residual_qq_plots.png" alt="Q-Q Plots">
            </div>
            <div class="plot">
                <h3>Prediction Scatter Plots</h3>
                <img src="prediction_scatter_plots.png" alt="Prediction Scatter Plots">
            </div>
        </div>
    </body>
    </html>
    """

    with open(os.path.join(output_dir, 'model_comparison.html'), 'w') as f:
        f.write(html_content)

    return {
        'f_statistic': f_stat,
        'p_value': p_value,
        'tukey_results': tukey,
        'model_stats': model_stats
    }

def main():
    print("Loading best models and results...")
    models_dict = load_best_models()

    print("\nCreating comparison plots...")
    plot_model_comparison(models_dict)

    print("\nGenerating detailed comparison...")
    print_detailed_comparison(models_dict)

    print("\nComparison plot saved as 'model_comparison.png'")

if __name__ == "__main__":
    # Load data
    print("Loading data...")
    data_dict = load_and_process_data(verbose=False)

    # Load model predictions and perform ANOVA
    print("\nPerforming model comparison analysis...")
    nam_preds, rf_preds, xgb_preds, y_true = load_model_predictions(data_dict)
    model_comparison = perform_model_anova(nam_preds, rf_preds, xgb_preds, y_true)

    print("\nANOVA Results:")
    print(f"F-statistic: {model_comparison['f_statistic']:.4f}")
    print(f"p-value: {model_comparison['p_value']:.4f}")

    print("\nModel Performance Metrics:")
    print(model_comparison['model_stats'][['Model', 'RMSE', 'MAE', 'R²']].to_string(index=False))

    print("\nTukey's HSD Test Results:")
    print(model_comparison['tukey_results'])

    print("\nDetailed analysis has been saved to the 'model_analysis' directory.")

In [ ]:

from analyze_database import load_and_process_data, FEATURE_DICT
from nam_analysis import NAM

def load_nam_model(results_dir):
    """Load the trained NAM model."""
    model_path = os.path.join(results_dir, 'best_model.pt')
    if not os.path.exists(model_path):
        raise FileNotFoundError("NAM model file not found")

    # Load with weights_only=False for compatibility
    checkpoint = torch.load(model_path, weights_only=False)
    model_state = checkpoint['model_state_dict']
    params = checkpoint['params']

    # Initialize model with same architecture
    model = NAM(num_features=len(FEATURE_DICT), hidden_layers=params['hidden_layers'])
    model.load_state_dict(model_state)
    model.eval()

    return model

def get_nam_feature_functions(model, X, feature_names, device='cpu'):
    """Extract individual feature functions from NAM."""
    model = model.to(device)
    X_tensor = torch.FloatTensor(X.to_numpy()).to(device)

    feature_functions = {}
    with torch.no_grad():
        for i, feature in enumerate(feature_names):
            # Get contribution for this feature across all samples
            feature_input = X_tensor[:, i].unsqueeze(1)
            subnet = model.subnets[i]
            contribution = subnet(feature_input).cpu().numpy()

            # Store feature values and their contributions
            feature_functions[feature] = {
                'values': X.iloc[:, i].values,
                'contributions': contribution.flatten()
            }

    return feature_functions

def load_shap_values():
    """Load pre-computed SHAP values."""
    shap_dir = 'shap_analysis_results'
    rf_shap = np.load(os.path.join(shap_dir, 'rf', 'shap_values.npy'))
    xgb_shap = np.load(os.path.join(shap_dir, 'xgb', 'shap_values.npy'))
    return rf_shap, xgb_shap

def compare_feature_importance(nam_functions, rf_shap, xgb_shap, feature_names, output_dir):
    """Compare feature importance across models."""
    # Calculate NAM importance (variance of contributions)
    nam_importance = []
    for feature in feature_names:
        contributions = nam_functions[feature]['contributions']
        importance = np.var(contributions)
        nam_importance.append(importance)

    # Calculate SHAP importance (mean absolute value)
    rf_importance = np.mean(np.abs(rf_shap), axis=0)
    xgb_importance = np.mean(np.abs(xgb_shap), axis=0)

    # Normalize importances
    nam_importance = nam_importance / np.sum(nam_importance)
    rf_importance = rf_importance / np.sum(rf_importance)
    xgb_importance = xgb_importance / np.sum(xgb_importance)

    # Create comparison plot
    plt.figure(figsize=(15, 8))
    x = np.arange(len(feature_names))
    width = 0.25

    plt.bar(x - width, nam_importance, width, label='NAM', color='green', alpha=0.7)
    plt.bar(x, rf_importance, width, label='Random Forest', color='blue', alpha=0.7)
    plt.bar(x + width, xgb_importance, width, label='XGBoost', color='red', alpha=0.7)

    plt.xlabel('Features')
    plt.ylabel('Normalized Importance')
    plt.title('Feature Importance Comparison Across Models')
    plt.xticks(x, feature_names, rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'importance_comparison.png'))
    plt.close()

    return nam_importance, rf_importance, xgb_importance

def compare_feature_effects(nam_functions, rf_shap, xgb_shap, X, feature_names, output_dir):
    """Compare how each model interprets feature effects."""
    key_features = ['M/Vlw', 'hw', 'ρsh', 'lw', 'b0']  # Most important features

    for feature in key_features:
        if feature not in feature_names:
            continue

        plt.figure(figsize=(15, 5))
        feature_idx = feature_names.index(feature)

        # Plot NAM function
        plt.subplot(1, 3, 1)
        values = nam_functions[feature]['values']
        contributions = nam_functions[feature]['contributions']
        order = np.argsort(values)
        plt.scatter(values[order], contributions[order], alpha=0.5, s=20)
        plt.plot(values[order], contributions[order], 'g-', alpha=0.7, label='NAM')
        plt.title(f'NAM Function\n{feature}')
        plt.xlabel('Feature Value')
        plt.ylabel('Contribution')

        # Plot RF SHAP values
        plt.subplot(1, 3, 2)
        shap.dependence_plot(
            feature_idx, rf_shap, X,
            feature_names=feature_names,
            show=False,
            ax=plt.gca()
        )
        plt.title(f'Random Forest SHAP\n{feature}')

        # Plot XGB SHAP values
        plt.subplot(1, 3, 3)
        shap.dependence_plot(
            feature_idx, xgb_shap, X,
            feature_names=feature_names,
            show=False,
            ax=plt.gca()
        )
        plt.title(f'XGBoost SHAP\n{feature}')

        plt.tight_layout()
        safe_feature = feature.replace('/', '_').replace('\\', '_')
        plt.savefig(os.path.join(output_dir, f'effect_comparison_{safe_feature}.png'))
        plt.close()

def analyze_directionality(nam_functions, rf_shap, xgb_shap, feature_names):
    """Analyze if models agree on feature effect directions."""
    directionality = {}

    for feature in feature_names:
        feature_idx = feature_names.index(feature)

        # NAM directionality (correlation between value and contribution)
        nam_values = nam_functions[feature]['values']
        nam_contrib = nam_functions[feature]['contributions']
        nam_corr = np.corrcoef(nam_values, nam_contrib)[0, 1]

        # SHAP directionality (correlation between value and SHAP value)
        rf_corr = np.corrcoef(nam_values, rf_shap[:, feature_idx])[0, 1]
        xgb_corr = np.corrcoef(nam_values, xgb_shap[:, feature_idx])[0, 1]

        directionality[feature] = {
            'nam': nam_corr,
            'rf': rf_corr,
            'xgb': xgb_corr,
            'agreement': np.sign(nam_corr) == np.sign(rf_corr) == np.sign(xgb_corr)
        }

    return directionality

def create_html_report(output_dir, feature_names, directionality, nam_imp, rf_imp, xgb_imp):
    """Create an HTML report with plots and descriptions."""
    # Format the lists of top features first
    nam_top = [feature_names[i] for i in np.argsort(nam_imp)[-5:][::-1]]
    rf_top = [feature_names[i] for i in np.argsort(rf_imp)[-5:][::-1]]
    xgb_top = [feature_names[i] for i in np.argsort(xgb_imp)[-5:][::-1]]

    html_content = f"""
    <html>
    <head>
        <title>Model Interpretability Comparison</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 40px; }}
            .plot-section {{ margin: 20px 0; padding: 20px; border: 1px solid #ddd; }}
            .plot {{ text-align: center; }}
            .description {{ margin: 20px 0; }}
            h2 {{ color: #2c3e50; }}
            .feature-section {{ margin: 10px 0; }}
            .model-comparison {{ display: flex; justify-content: space-around; }}
            .model-box {{
                flex: 1;
                margin: 10px;
                padding: 15px;
                border: 1px solid #ddd;
                border-radius: 5px;
            }}
        </style>
    </head>
    <body>
        <h1>Model Interpretability Comparison Report</h1>

        <div class="plot-section">
            <h2>1. Overall Feature Importance Comparison</h2>
            <div class="plot">
                <img src="importance_comparison.png" alt="Feature Importance Comparison">
            </div>
            <div class="description">
                <p>This plot compares how each model ranks feature importance. Key observations:</p>
                <ul>
                    <li>NAM top 5: {nam_top}</li>
                    <li>RF top 5: {rf_top}</li>
                    <li>XGB top 5: {xgb_top}</li>
                </ul>
            </div>
        </div>

        <div class="plot-section">
            <h2>2. Individual Feature Effects</h2>
            <p>For key features, we compare how each model interprets their effects:</p>
    """

    # Add key feature comparisons
    key_features = ['M/Vlw', 'hw', 'ρsh', 'lw', 'b0']
    for feature in key_features:
        if feature in feature_names:
            safe_feature = feature.replace('/', '_').replace('\\', '_')
            html_content += f"""
            <div class="feature-section">
                <h3>Feature: {feature}</h3>
                <div class="plot">
                    <img src="effect_comparison_{safe_feature}.png" alt="Effect Comparison for {feature}">
                </div>
                <div class="description">
                    <p>Direction agreement: {'Yes' if directionality[feature]['agreement'] else 'No'}</p>
                    <p>Effect direction:</p>
                    <ul>
                        <li>NAM: {'positive' if directionality[feature]['nam'] > 0 else 'negative'}</li>
                        <li>RF: {'positive' if directionality[feature]['rf'] > 0 else 'negative'}</li>
                        <li>XGB: {'positive' if directionality[feature]['xgb'] > 0 else 'negative'}</li>
                    </ul>
                </div>
            </div>
            """

    # Add model comparison section
    html_content += """
        </div>

        <div class="plot-section">
            <h2>3. Model Comparison Summary</h2>
            <div class="model-comparison">
                <div class="model-box">
                    <h3>NAM</h3>
                    <ul>
                        <li>Direct interpretation of feature effects</li>
                        <li>Smooth, continuous functions</li>
                        <li>No interaction effects</li>
                        <li>Best for initial understanding</li>
                    </ul>
                </div>
                <div class="model-box">
                    <h3>SHAP (RF & XGB)</h3>
                    <ul>
                        <li>Captures feature interactions</li>
                        <li>Local and global interpretability</li>
                        <li>More complex relationships</li>
                        <li>Best for detailed analysis</li>
                    </ul>
                </div>
            </div>
        </div>

        <div class="plot-section">
            <h2>4. Engineering Recommendations</h2>
            <ul>
                <li>Use NAM for initial design decisions and understanding individual parameter effects</li>
                <li>Use SHAP for understanding complex interactions and verifying NAM findings</li>
                <li>Pay special attention to features where models disagree</li>
                <li>Consider both approaches for comprehensive understanding</li>
            </ul>
        </div>
    </body>
    </html>
    """

    # Write HTML report
    with open(os.path.join(output_dir, 'interpretability_report.html'), 'w') as f:
        f.write(html_content)

def main():
    # Create output directory
    output_dir = 'interpretability_comparison'
    os.makedirs(output_dir, exist_ok=True)

    # Load data
    print("Loading data...")
    data_dict = load_and_process_data(verbose=False)
    X_test = data_dict['X_test']
    feature_names = data_dict['feature_names']

    # Find model directories
    nam_dirs = [d for d in os.listdir('.') if d.startswith('nam_results_')]
    if not nam_dirs:
        raise FileNotFoundError("No NAM results directory found")
    latest_nam_dir = max(nam_dirs)

    # Load models and compute interpretations
    print("Loading models and computing interpretations...")
    nam_model = load_nam_model(latest_nam_dir)
    nam_functions = get_nam_feature_functions(nam_model, X_test, feature_names)
    rf_shap, xgb_shap = load_shap_values()

    # Compare feature importance
    print("\nComparing feature importance...")
    nam_imp, rf_imp, xgb_imp = compare_feature_importance(
        nam_functions, rf_shap, xgb_shap, feature_names, output_dir
    )

    # Compare feature effects
    print("Comparing feature effects...")
    compare_feature_effects(
        nam_functions, rf_shap, xgb_shap, X_test, feature_names, output_dir
    )

    # Analyze directionality
    print("Analyzing effect directionality...")
    directionality = analyze_directionality(
        nam_functions, rf_shap, xgb_shap, feature_names
    )

    # Create HTML report
    print("\nGenerating HTML report...")
    create_html_report(output_dir, feature_names, directionality, nam_imp, rf_imp, xgb_imp)

    # Print insights
    print("\nKey Insights:")
    print("\n1. Feature Importance Agreement:")
    # Get top 5 features by each model
    top_5_nam = np.argsort(nam_imp)[-5:]
    top_5_rf = np.argsort(rf_imp)[-5:]
    top_5_xgb = np.argsort(xgb_imp)[-5:]

    print("\nTop 5 Important Features:")
    print("NAM:", [feature_names[i] for i in top_5_nam[::-1]])
    print("RF: ", [feature_names[i] for i in top_5_rf[::-1]])
    print("XGB:", [feature_names[i] for i in top_5_xgb[::-1]])

    # Find features where all models agree on direction
    print("\n2. Directional Agreement:")
    agreement_features = [f for f, d in directionality.items() if d['agreement']]
    print("\nFeatures where all models agree on effect direction:")
    for feature in agreement_features:
        direction = "positive" if directionality[feature]['nam'] > 0 else "negative"
        print(f"- {feature}: {direction} effect")

    # Find features with disagreement
    disagreement_features = [f for f, d in directionality.items() if not d['agreement']]
    print("\nFeatures with disagreement in effect direction:")
    for feature in disagreement_features:
        print(f"\n{feature}:")
        print(f"  NAM: {'positive' if directionality[feature]['nam'] > 0 else 'negative'}")
        print(f"  RF:  {'positive' if directionality[feature]['rf'] > 0 else 'negative'}")
        print(f"  XGB: {'positive' if directionality[feature]['xgb'] > 0 else 'negative'}")

    print(f"\nDetailed report with plots has been generated at: {output_dir}/interpretability_report.html")

if __name__ == "__main__":
    main()